<a href="https://colab.research.google.com/github/lucassant0s/data-science/blob/main/science/_7DaysOfCode_Despesas_CEAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from functools import reduce
import os
import requests
import csv
import glob
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import plot, iplot
import plotly.express as px
import cufflinks as cf # para conectar o plotly ao pandas
from plotly.subplots import make_subplots
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)

### Criação das URLs com base na lista de anos (assim poderemos fazer o download das bases de dados de forma dinâmica)

#### Removendo arquivos existentes

In [ ]:
files = glob.glob('/content/drive/MyDrive/estudo-data-science/7-days-of-code/*')
for f in files:
  os.remove(f)

#### Download das bases de dados, savando no Google Drive dentro

In [ ]:
years = [2022, 2021, 2020, 2019, 2018]

for year in years:
  req = requests.get('https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_{}.csv'.format(year))
  # Convert to iterator by splitting on \n chars
  lines = req.text.splitlines()
  # Parse as CSV object
  reader = csv.reader(lines, delimiter=';')
  next(reader)
  with open('/content/drive/MyDrive/estudo-data-science/7-days-of-code/despesa_ceaps_{}.csv'.format(year), mode='w') as file:
    writer = csv.writer(file, delimiter=';')
    for row in reader:
      writer.writerow(row)

#### Carregar todas as bases de dados

In [ ]:
files_list = []
df_years = []

for dirname, _, filenames in os.walk('/content/drive/MyDrive/estudo-data-science/7-days-of-code/'):
  for filename in filenames:
    print(os.path.join(dirname, filename))
    
    file = os.path.join(dirname, filename)
    fileExt = file[58:-4]
    print(fileExt)
    #read each file in their variable name
    exec('data_{:s} = pd.read_csv("{:s}", delimiter=";")'.format(fileExt, file))
    exec('df_years.append(pd.read_csv("{:s}", delimiter=";"))'.format(file))
    
    #append list of files
    files_list.append('data_{:s}'.format(fileExt))

    # -- Display the files that will be loaded and the varible names
    # print('data_{}'.format(fileExt))

files_list
df_years

/content/drive/MyDrive/estudo-data-science/7-days-of-code/despesa_ceaps_2022.csv
despesa_ceaps_2022
/content/drive/MyDrive/estudo-data-science/7-days-of-code/despesa_ceaps_2021.csv
despesa_ceaps_2021
/content/drive/MyDrive/estudo-data-science/7-days-of-code/despesa_ceaps_2020.csv
despesa_ceaps_2020
/content/drive/MyDrive/estudo-data-science/7-days-of-code/despesa_ceaps_2019.csv
despesa_ceaps_2019
/content/drive/MyDrive/estudo-data-science/7-days-of-code/despesa_ceaps_2018.csv
despesa_ceaps_2018


[       ANO  MES           SENADOR  \
 0     2022    1      ACIR GURGACZ   
 1     2022    1      ACIR GURGACZ   
 2     2022    1      ACIR GURGACZ   
 3     2022    1      ACIR GURGACZ   
 4     2022    1      ACIR GURGACZ   
 ...    ...  ...               ...   
 5268  2022    4  ZEQUINHA MARINHO   
 5269  2022    4  ZEQUINHA MARINHO   
 5270  2022    4  ZEQUINHA MARINHO   
 5271  2022    4  ZEQUINHA MARINHO   
 5272  2022    4  ZEQUINHA MARINHO   
 
                                            TIPO_DESPESA            CNPJ_CPF  \
 0     Aluguel de imóveis para escritório político, c...      004.948.028-63   
 1                   Divulgação da atividade parlamentar  26.320.603/0001-64   
 2                   Divulgação da atividade parlamentar  13.659.201/0001-47   
 3                   Divulgação da atividade parlamentar  23.652.846/0001-01   
 4                   Divulgação da atividade parlamentar  08.941.827/0001-01   
 ...                                                 ...      

#### Vamos juntar todos os datasets que nos criamos sobre as despesas da CEAPS

In [ ]:
df_despesa_ceaps = pd.concat(df_years)

Verificando a quantidade total dos dataframes

In [ ]:
data_despesa_ceaps_2022.shape[0] + data_despesa_ceaps_2021.shape[0] + data_despesa_ceaps_2020.shape[0] + data_despesa_ceaps_2019.shape[0] + data_despesa_ceaps_2018.shape[0]

82316

Verificando o total do dataframe concatenando todos os outros dataframes separados por ANOS

In [ ]:
df_despesa_ceaps.shape[0]

82316

#### Imprimindo os primeiros registro do nosso dataframe

In [ ]:
df_despesa_ceaps.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,18/01/2022,Divulgação da atividade parlamentar,1000,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,17/01/2022,Divulgação da atividade parlamentar,2000,2173617


#### Verificando todos os nossos tipos de dados

In [ ]:
df_despesa_ceaps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82316 entries, 0 to 24491
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                82316 non-null  int64 
 1   MES                82316 non-null  int64 
 2   SENADOR            82316 non-null  object
 3   TIPO_DESPESA       82316 non-null  object
 4   CNPJ_CPF           82316 non-null  object
 5   FORNECEDOR         82316 non-null  object
 6   DOCUMENTO          79018 non-null  object
 7   DATA               82316 non-null  object
 8   DETALHAMENTO       51167 non-null  object
 9   VALOR_REEMBOLSADO  82316 non-null  object
 10  COD_DOCUMENTO      82316 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 7.5+ MB


### Data Wrangling

#### TO-DO

- Converter valores com tipos incorretos (datas e valores monetários)
- Remover registros com valores nulos ou inexistentes
- Remover valores duplicados
- Remover formatação da coluna de CPNJ_CPF
- Renomear o nome coluna de CPNJ_CPF para um nome das generico
- Criar uma coluna para identificar o tipo de documento (CPNJ OU CPF)

#### Verificando e removendo dados nulos

In [ ]:
df_despesa_ceaps.isnull().sum()

ANO                      0
MES                      0
SENADOR                  0
TIPO_DESPESA             0
CNPJ_CPF                 0
FORNECEDOR               0
DOCUMENTO             3298
DATA                     0
DETALHAMENTO         31149
VALOR_REEMBOLSADO        0
COD_DOCUMENTO            0
dtype: int64

#### Como só encontramos dados nulos nas colunas Documento e Detalhamento, podemos adicionar uma label "Unknown" nesses registro faltantes

In [ ]:
df_despesa_ceaps.fillna('Unknown', inplace=True)

#### Agora verificaremos se ainda temos registros com valores nulos depois do processo de substituição

In [ ]:
df_despesa_ceaps.isnull().sum()

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DOCUMENTO            0
DATA                 0
DETALHAMENTO         0
VALOR_REEMBOLSADO    0
COD_DOCUMENTO        0
dtype: int64

#### Vamos verificar se temos registros duplicados em nosso dataframe

In [ ]:
print('Quantidade de valores duplicados = {}, pelo que podemos ver não temos a necessidade de remover valores duplicados.'.format(df_despesa_ceaps.duplicated(keep='first').sum()))

Quantidade de valores duplicados = 0, pelo que podemos ver não temos a necessidade de remover valores duplicados.


#### Renomear os nomes das colunas para que fiquem mais acessíveis

In [ ]:
columns = [column.lower() for column in df_despesa_ceaps.columns]

In [ ]:
columns

['ano',
 'mes',
 'senador',
 'tipo_despesa',
 'cnpj_cpf',
 'fornecedor',
 'documento',
 'data',
 'detalhamento',
 'valor_reembolsado',
 'cod_documento']

In [ ]:
df_despesa_ceaps.columns = columns

In [ ]:
df_despesa_ceaps.head()

,ano,mes,senador,tipo_despesa,cnpj_cpf,fornecedor,documento,data,detalhamento,valor_reembolsado,cod_documento
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,18/01/2022,Divulgação da atividade parlamentar,1000,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,17/01/2022,Divulgação da atividade parlamentar,2000,2173617


#### Conversão de datas

In [ ]:
df_despesa_ceaps['data'] = pd.to_datetime(df_despesa_ceaps['data'], format='%d/%m/%Y', errors='coerce')

In [ ]:
df_despesa_ceaps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82316 entries, 0 to 24491
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ano                82316 non-null  int64         
 1   mes                82316 non-null  int64         
 2   senador            82316 non-null  object        
 3   tipo_despesa       82316 non-null  object        
 4   cnpj_cpf           82316 non-null  object        
 5   fornecedor         82316 non-null  object        
 6   documento          82316 non-null  object        
 7   data               82311 non-null  datetime64[ns]
 8   detalhamento       82316 non-null  object        
 9   valor_reembolsado  82316 non-null  object        
 10  cod_documento      82316 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 7.5+ MB


In [ ]:
df_despesa_ceaps.head()

,ano,mes,senador,tipo_despesa,cnpj_cpf,fornecedor,documento,data,detalhamento,valor_reembolsado,cod_documento
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,2022-01-03,Despesa com pagamento de aluguel de imóvel par...,6000,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,2022-01-04,Despesa com divulgação da atividade parlamenta...,1500,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,2022-01-14,Despesa com produção de texto e edição de víde...,6000,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,2022-01-18,Divulgação da atividade parlamentar,1000,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,2022-01-17,Divulgação da atividade parlamentar,2000,2173617


#### Convertendo valores reembolsados **str => int**

In [ ]:
valor_reembolsado = df_despesa_ceaps['valor_reembolsado']

In [ ]:
valor_reembolsado = valor_reembolsado.replace(r'[^\w\s]|_', '.', regex=True)

In [ ]:
df_despesa_ceaps['valor_reembolsado'] = valor_reembolsado.astype('float')

In [ ]:
df_despesa_ceaps['valor_reembolsado'].astype('float')

0        6000.00
1        1500.00
2        6000.00
3        1000.00
4        2000.00
          ...   
24487     220.05
24488      73.02
24489     232.01
24490    1132.69
24491    1260.45
Name: valor_reembolsado, Length: 82316, dtype: float64

Vamos remover as macaras dos dados de CPF e CNPJ, depois vamos renomear o nome da coluna e adicionar uma coluna para identificamos o que é CPF e CNPJ

In [ ]:
cnpj_cpf = df_despesa_ceaps['cnpj_cpf']

In [ ]:
cnpj_cpf = cnpj_cpf.replace(r'[^\w\s]|_', '', regex=True)
cnpj_cpf

0           00494802863
1        26320603000164
2        13659201000147
3        23652846000101
4        08941827000101
              ...      
24487    16979906000130
24488    16979906000130
24489    16979906000130
24490    16978175000108
24491    16978175000108
Name: cnpj_cpf, Length: 82316, dtype: object

In [ ]:
df_despesa_ceaps['cnpj_cpf'] = cnpj_cpf

In [ ]:
cnpj_cpf_length = df_despesa_ceaps['cnpj_cpf'].str.len()

Criação da coluna "tipo_documento"

In [ ]:
df_despesa_ceaps['tipo_documento'] = cnpj_cpf_length.map(lambda value: 'cpf' if value == 11 else 'cnpj')

Removendo colunas que não contém informações relevantes

In [ ]:
df_despesa_ceaps.drop(columns=['ano', 'mes', 'documento', 'cod_documento'], inplace=True)

Renomeando a coluna "cnpj_cpf" => "documento"

In [ ]:
df_despesa_ceaps.rename(columns={'cnpj_cpf': 'documento'}, inplace=True)

In [ ]:
df_despesa_ceaps.head()

,senador,tipo_despesa,documento,fornecedor,data,detalhamento,valor_reembolsado,tipo_documento
0,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",00494802863,GILBERTO PISELO DO NASCIMENTO,2022-01-03,Despesa com pagamento de aluguel de imóvel par...,6000.0,cpf
1,ACIR GURGACZ,Divulgação da atividade parlamentar,26320603000164,INFORMANAHORA,2022-01-04,Despesa com divulgação da atividade parlamenta...,1500.0,cnpj
2,ACIR GURGACZ,Divulgação da atividade parlamentar,13659201000147,LINHA PURPURA FOTO E VIDEO LTDA,2022-01-14,Despesa com produção de texto e edição de víde...,6000.0,cnpj
3,ACIR GURGACZ,Divulgação da atividade parlamentar,23652846000101,ROBERTO GUTIERREZ DA ROCHA M.E.I.,2022-01-18,Divulgação da atividade parlamentar,1000.0,cnpj
4,ACIR GURGACZ,Divulgação da atividade parlamentar,08941827000101,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,2022-01-17,Divulgação da atividade parlamentar,2000.0,cnpj


Notamos que temos valores fora dos ranges especificado em cada CSV, removeremos esses valores para que eles não desbaleem as nossas análises

In [ ]:
df_despesa_ceaps[df_despesa_ceaps['data'].dt.year < 2019].count()

senador              24482
tipo_despesa         24482
documento            24482
fornecedor           24482
data                 24482
detalhamento         24482
valor_reembolsado    24482
tipo_documento       24482
dtype: int64

Verificando o tamanho atual do DataFrame

In [ ]:
df_despesa_ceaps.shape[0]

82316

Removemos valores fora do escopo da análise

In [ ]:
df_despesa_ceaps.drop(df_despesa_ceaps[df_despesa_ceaps['data'].dt.year < 2018].index, inplace=True)
df_despesa_ceaps.shape[0]

81583

### DataFrame com ESTADOS, COTAS e nome dos SENADORES

#### DataFrame dos estados brasileiros

Informações sobre as cotas foram encontradas no site https://www12.senado.leg.br/transparencia/leg/pdf/CotaExercicioAtivParlamSenadores.pdf

In [ ]:
states = {
  'uf': ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO'],
  'estados': ['Acre','Alagoas','Amapá','Amazonas','Bahia','Ceará','Distrito Federal','Espírito Santo','Goiás','Maranhão','Mato Grosso','Mato Grosso do Sul','Minas Gerais','Pará','Paraíba','Paraná','Pernambuco','Piauí','Rio de Janeiro','Rio Grande do Norte','Rio Grande do Sul','Rondônia','Roraima','Santa Catarina','São Paulo','Sergipe','Tocantins'],
  'cotas': [38854.45,35056.20,42855.20, 44276.60, 35416.20, 38186.60, 21045.20, 33176.60, 21045.20,37396.60,34934.45,32905.20,28496.20,40426.20,35555.20,32586.60,36266.60,38834.45,31816.20,35976.20,35886.60,34615.20,40724.45,32871.32,30226.20,41844.45,25215.20]
}
df_states = pd.DataFrame(states)
df_states.head(5)

,uf,estados,cotas
0,AC,Acre,38854.45
1,AL,Alagoas,35056.20
2,AP,Amapá,42855.20
3,AM,Amazonas,44276.60
4,BA,Bahia,35416.20


In [ ]:
df_states['cotas_ano'] = df_states['cotas'] * 12
df_states.head(5)

,uf,estados,cotas,cotas_ano
0,AC,Acre,38854.45,466253.4
1,AL,Alagoas,35056.20,420674.4
2,AP,Amapá,42855.20,514262.4
3,AM,Amazonas,44276.60,531319.2
4,BA,Bahia,35416.20,424994.4


#### DataFrame com nomes dos senadores e seus estados

Por algum motivo a função **drop** não reconheceu a coluna **endereco**, então usando uma segunda estratégia para fazer o slice do DataFrame

In [ ]:
df_parla_inter = pd.read_csv('/content/drive/MyDrive/estudo-data-science/parla_inter_csv.csv', delimiter=';', encoding='ISO-8859-1')
columns_parla_inter = [column.lower() for column in df_parla_inter.columns]
df_parla_inter.columns = columns_parla_inter
df_parla_inter.drop(columns=['partido', 'titularidade', 'mandato', 'telefones', 'dtnasc', 'email', 'chefe_gab'], inplace=True)
df_parla_inter = df_parla_inter[['nome_parlamentar', 'uf']]
df_parla_inter.columns = ['senador', 'uf']
df_parla_inter.head(5)

,senador,uf
0,ACIR GURGACZ,RO
1,ALESSANDRO VIEIRA,SE
2,ALEXANDRE SILVEIRA,MG
3,ALVARO DIAS,PR
4,ANGELO CORONEL,BA


Agora temos uma possibilidade para verificar o quanto cada senador utilizou do limite das cotas disponibilizadas pelos por estado.

### Data Visualization & Storytelling

#### Qual foi o senador que mais declarou despesas?

Contagem durante todo o mandato (2019 - 2022) com os dados que nós temos até o presente momento

In [ ]:
df_despesa_ceaps['senador'].value_counts()

WELLINGTON FAGUNDES     2593
HUMBERTO COSTA          2175
PAULO PAIM              2047
CARLOS VIANA            1849
JOSÉ SERRA              1765
                        ... 
NEY SUASSUNA              15
SIQUEIRA CAMPOS            4
FABIO GARCIA               4
VIRGINIO DE CARVALHO       3
NAILDE PANTA               1
Name: senador, Length: 158, dtype: int64

In [ ]:
print('Os gastos do senador Carlos Viana representão {}% do total de gastos declarado durando o último mandato (2019 - 2022) até o presente momento dessa análise.'.format(((df_despesa_ceaps['senador'].value_counts()[0] / df_despesa_ceaps.shape[0]) * 100).round(2)))

Os gastos do senador Carlos Viana representão 3.18% do total de gastos declarado durando o último mandato (2019 - 2022) até o presente momento dessa análise.


#### Qual foi o percentual da cota que os senadores gastaram?

Para fazer os calculos relacionados com as cotas de cada senador, tivemos que fazer mais uma etapa de Data Wrangling

In [ ]:
years

for year in years:
  exec('valor_total_{} = df_despesa_ceaps[df_despesa_ceaps["data"].dt.year == {}]["valor_reembolsado"].sum()'.format(year, year))

In [ ]:
years

for year in years:
  exec('valor_total_senador_{} = df_despesa_ceaps[df_despesa_ceaps["data"].dt.year == {}].groupby("senador")["valor_reembolsado"].agg(valor_total=sum)'.format(year, year))
  exec('valor_total_senador_{}.reset_index(inplace=True)'.format(year))

In [ ]:
valor_total_senador_2021[valor_total_senador_2021['senador'] == 'DAVI ALCOLUMBRE'] = 512570.92

#### Merge com os DataFrames de estados e cotas

In [ ]:
years

for year in years:
  exec('valor_total_senador_{} = pd.merge(valor_total_senador_{}, df_parla_inter, how="left", on="senador")'.format(year, year))
  exec('valor_total_senador_{} = pd.merge(valor_total_senador_{}, df_states, how="left", on="uf")'.format(year, year))

In [ ]:
valor_total_senador_2021.head(5)

,senador,valor_total,uf,estados,cotas,cotas_ano
0,ACIR GURGACZ,404456.58,RO,Rondônia,34615.20,415382.4
1,ALESSANDRO VIEIRA,420529.52,SE,Sergipe,41844.45,502133.4
2,ALVARO DIAS,27231.37,PR,Paraná,32586.60,391039.2
3,ANGELO CORONEL,438972.87,BA,Bahia,35416.20,424994.4
4,ANTONIO ANASTASIA,292603.46,NaN,NaN,NaN,NaN


#### Removendo os senadores que estão sem informações de estado e cotas

In [ ]:
valor_total_senador_2021.isnull().sum()

senador         0
valor_total     0
uf             17
estados        17
cotas          17
cotas_ano      17
dtype: int64

In [ ]:
years

for year in years:
  exec('valor_total_senador_{}.dropna(inplace=True)'.format(year, year))

In [ ]:
valor_total_senador_2021.isnull().sum()

senador        0
valor_total    0
uf             0
estados        0
cotas          0
cotas_ano      0
dtype: int64

#### Calculo de porcetagens das cotas que os senadores consumiram durante os anos dos seus mandatos

In [ ]:
years

for year in years:
  exec('valor_total_senador_{}["por_cota"] = ((valor_total_senador_{}["valor_total"] / valor_total_senador_{}["cotas_ano"]) * 100).round(2)'.format(year, year, year))

In [ ]:
valor_total_senador_2021.sort_values(by='valor_total', ascending=False).head(5)

,senador,valor_total,uf,estados,cotas,cotas_ano,por_cota
59,OMAR AZIZ,527681.31,AM,Amazonas,44276.60,531319.2,99.32
70,ROGÉRIO CARVALHO,504670.23,SE,Sergipe,41844.45,502133.4,100.51
54,MECIAS DE JESUS,501632.05,RR,Roraima,40724.45,488693.4,102.65
42,LUCAS BARRETO,493871.53,AP,Amapá,42855.20,514262.4,96.03
78,TELMÁRIO MOTA,491701.38,RR,Roraima,40724.45,488693.4,100.62


In [ ]:
valor_total_senador_2018_contagem = valor_total_senador_2018[valor_total_senador_2018['por_cota'] >= 90]['por_cota'].count()
valor_total_senador_2019_contagem = valor_total_senador_2019[valor_total_senador_2019['por_cota'] >= 90]['por_cota'].count()
valor_total_senador_2020_contagem = valor_total_senador_2020[valor_total_senador_2020['por_cota'] >= 90]['por_cota'].count()
valor_total_senador_2021_contagem = valor_total_senador_2021[valor_total_senador_2021['por_cota'] >= 90]['por_cota'].count()
valor_total_senador_2022_contagem = valor_total_senador_2022[valor_total_senador_2022['por_cota'] >= 90]['por_cota'].count()

In [ ]:
print('Quantidade de senadores que gastaram mais de 90% da cota anual em 2018: {}, isso corresponde a {}% do nossa dataframe.'.format(valor_total_senador_2018_contagem, ((valor_total_senador_2018_contagem / valor_total_senador_2018.shape[0])*100).round(2)))
print('Quantidade de senadores que gastaram mais de 90% da cota anual em 2019: {}, isso corresponde a {}% do nossa dataframe.'.format(valor_total_senador_2019_contagem, ((valor_total_senador_2019_contagem / valor_total_senador_2019.shape[0])*100).round(2)))
print('Quantidade de senadores que gastaram mais de 90% da cota anual em 2020: {}, isso corresponde a {}% do nossa dataframe.'.format(valor_total_senador_2020_contagem, ((valor_total_senador_2019_contagem / valor_total_senador_2020.shape[0])*100).round(2)))
print('Quantidade de senadores que gastaram mais de 90% da cota anual em 2021: {}, isso corresponde a {}% do nossa dataframe.'.format(valor_total_senador_2021_contagem, ((valor_total_senador_2019_contagem / valor_total_senador_2021.shape[0])*100).round(2)))
print('Quantidade de senadores que gastaram mais de 90% da cota anual em 2022: {} // Estamos no mês de maio, logo, não temos como avaliar essa métrica.'.format(valor_total_senador_2022_contagem))

Quantidade de senadores que gastaram mais de 90% da cota anual em 2018: 8, isso corresponde a 25.81% do nossa dataframe.
Quantidade de senadores que gastaram mais de 90% da cota anual em 2019: 26, isso corresponde a 40.0% do nossa dataframe.
Quantidade de senadores que gastaram mais de 90% da cota anual em 2020: 14, isso corresponde a 39.39% do nossa dataframe.
Quantidade de senadores que gastaram mais de 90% da cota anual em 2021: 28, isso corresponde a 37.68% do nossa dataframe.
Quantidade de senadores que gastaram mais de 90% da cota anual em 2022: 0 // Estamos no mês de maio, logo, não temos como avaliar essa métrica.


#### Quantidade de gastos declarados por anos, para avaliar se a quantidade de gastos declarados influência diretamente na proporção de senadores ultrapassando 90% das cotas anuais

In [ ]:
years

for year in years:
  exec('valor_total_{}_contagem_gastos = df_despesa_ceaps[df_despesa_ceaps["data"].dt.year == {}].shape[0]'.format(year, year))

print('Quantidade de gastos declarado no ano 2018: {}'.format(valor_total_2018_contagem_gastos))
print('Quantidade de gastos declarado no ano 2019: {}'.format(valor_total_2019_contagem_gastos))
print('Quantidade de gastos declarado no ano 2020: {}'.format(valor_total_2020_contagem_gastos))
print('Quantidade de gastos declarado no ano 2021: {}'.format(valor_total_2021_contagem_gastos))
print('Quantidade de gastos declarado no ano 2022: {}'.format(valor_total_2022_contagem_gastos))

Quantidade de gastos declarado no ano 2018: 24252
Quantidade de gastos declarado no ano 2019: 21508
Quantidade de gastos declarado no ano 2020: 13872
Quantidade de gastos declarado no ano 2021: 16686
Quantidade de gastos declarado no ano 2022: 5260


#### Quais foram as categorias de gastos mais declaradas?

In [ ]:
years

for year in years:
  exec('valor_total_{}_categoria = df_despesa_ceaps[df_despesa_ceaps["data"].dt.year == {}]["tipo_despesa"].value_counts()'.format(year, year))

In [ ]:
valor_total_2018_categoria = valor_total_2018_categoria.to_frame().reset_index().rename(columns={'index': 'despesa'})
valor_total_2019_categoria = valor_total_2019_categoria.to_frame().reset_index().rename(columns={'index': 'despesa'})
valor_total_2020_categoria = valor_total_2020_categoria.to_frame().reset_index().rename(columns={'index': 'despesa'})
valor_total_2021_categoria = valor_total_2021_categoria.to_frame().reset_index().rename(columns={'index': 'despesa'})
valor_total_2022_categoria = valor_total_2022_categoria.to_frame().reset_index().rename(columns={'index': 'despesa'})

### Análise dos gastos por ano/mês

#### Criando DataFrame agrupados por ano / mês / despesa

In [ ]:
df_despesa_ceaps_by_ano_mes_despesa = df_despesa_ceaps.groupby([df_despesa_ceaps["data"].dt.year.rename('ano'), df_despesa_ceaps["data"].dt.month.rename('mes'), 'tipo_despesa'])['valor_reembolsado'].agg(valor_total=sum)
df_despesa_ceaps_by_ano_mes_despesa = df_despesa_ceaps_by_ano_mes_despesa.reset_index()
df_despesa_ceaps_by_ano_mes_despesa['ano'] = df_despesa_ceaps_by_ano_mes_despesa['ano'].astype('int')
df_despesa_ceaps_by_ano_mes_despesa['mes'] = df_despesa_ceaps_by_ano_mes_despesa['mes'].astype('int')
df_despesa_ceaps_by_ano_mes_despesa.head()

,ano,mes,tipo_despesa,valor_total
0,2018,1,"Aluguel de imóveis para escritório político, c...",327869.75
1,2018,1,Aquisição de material de consumo para uso no e...,57264.06
2,2018,1,"Contratação de consultorias, assessorias, pesq...",189266.37
3,2018,1,Divulgação da atividade parlamentar,172204.16
4,2018,1,"Locomoção, hospedagem, alimentação, combustíve...",333042.54


In [ ]:
df_despesa_ceaps_by_ano_mes_despesa['mes'] = df_despesa_ceaps_by_ano_mes_despesa['mes'].map({ 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun', 7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Des' })

#### Criando DataFrame agrupados por ano / mês / senador

In [ ]:
df_despesa_ceaps_by_ano_mes_senador = df_despesa_ceaps.groupby([df_despesa_ceaps["data"].dt.year.rename('ano'), df_despesa_ceaps["data"].dt.month.rename('mes'), 'senador'])['valor_reembolsado'].agg(valor_total=sum)
df_despesa_ceaps_by_ano_mes_senador = df_despesa_ceaps_by_ano_mes_senador.reset_index()
df_despesa_ceaps_by_ano_mes_senador['ano'] = df_despesa_ceaps_by_ano_mes_senador['ano'].astype('int')
df_despesa_ceaps_by_ano_mes_senador['mes'] = df_despesa_ceaps_by_ano_mes_senador['mes'].astype('int')
df_despesa_ceaps_by_ano_mes_senador[df_despesa_ceaps_by_ano_mes_senador['ano'] == 2019].sort_values(by='valor_total', ascending=False).head()

,ano,mes,senador,valor_total
1826,2019,12,ANGELO CORONEL,286868.42
1525,2019,8,DÁRIO BERGER,162962.15
1189,2019,3,ROSE DE FREITAS,125095.07
1814,2019,11,SÉRGIO PETECÃO,87680.48
1135,2019,3,DAVI ALCOLUMBRE,85429.35


### Utilizando Potly

#### Visualização dos GASTOS com DESPESAS

In [ ]:
df_despesa_ceaps.tail()

,senador,tipo_despesa,documento,fornecedor,data,detalhamento,valor_reembolsado,tipo_documento
24487,ZEZÉ PERRELLA,"Locomoção, hospedagem, alimentação, combustíve...",16979906000130,Posto Vila da Serra Ltda,2018-12-13,Unknown,220.05,cnpj
24488,ZEZÉ PERRELLA,"Locomoção, hospedagem, alimentação, combustíve...",16979906000130,Posto Vila da Serra Ltda,2018-12-19,Unknown,73.02,cnpj
24489,ZEZÉ PERRELLA,"Locomoção, hospedagem, alimentação, combustíve...",16979906000130,Posto Vila da Serra Ltda,2018-12-19,Unknown,232.01,cnpj
24490,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",16978175000108,Adria Viagens e Turismo Ltda,2018-12-05,"Companhia Aérea: AZUL, Localizador: CI5PSK. Pa...",1132.69,cnpj
24491,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",16978175000108,Adria Viagens e Turismo Ltda,2018-12-17,"Companhia Aérea: LATAM, Localizador: OHNRAG. P...",1260.45,cnpj


In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Locomoção, hospedagem, alimentação, combustíveis e lubrificantes",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Passagens aéreas, aquáticas e terrestres nacionais"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Passagens aéreas, aquáticas e terrestres nacionais",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles."))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Aluguel de imóveis para escritório político",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Aquisição de material de consumo para uso no escritório político"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Aquisição de material de consumo para uso no escritório político",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Contratação de consultorias, assessorias"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Contratação de consultorias",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Divulgação da atividade parlamentar"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Divulgação da atividade parlamentar",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

In [ ]:
fig_despesa = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'))
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2018) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=1, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2019) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=2, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2020) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=3, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2021) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=4, col=1)
fig_despesa.add_trace(go.Scatter(x=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['mes'],
                    y=df_despesa_ceaps_by_ano_mes_despesa.query('(ano == 2022) and (tipo_despesa.str.contains("Serviços de Segurança Privada"))', engine='python')['valor_total'],
                    line_shape='linear', hovertemplate='Total: R$ %{y:,.2f}'+'<br>Mês: %{x}', name=""), row=5, col=1)

fig_despesa.update_xaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_yaxes(title_font_size=18, tickfont_size=16)
fig_despesa.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Serviços de Segurança Privada",
    hoverlabel_font_size=14,
    showlegend=False,
    height=700, 
    width=1000,
    font=dict(
        size=18,
        color="RebeccaPurple"
    )
)
fig_despesa.show(renderer='colab')

#### Visualiação dos GASTOS por SENADORES

In [ ]:
fig_senadores = make_subplots(rows=5, cols=1, subplot_titles=('2018', '2019', '2020', '2021', '2022'), shared_yaxes=True)
fig_senadores.add_trace(go.Bar(
    x=valor_total_senador_2018.sort_values(by='valor_total', ascending=False).head(10)['senador'],
    y=valor_total_senador_2018.sort_values(by='valor_total', ascending=False).head(10)['valor_total'],
    marker_color='purple', name="", hovertemplate='R$ %{y:,.2f}', texttemplate='R$ %{y:,.2s}'
), row=1, col=1)
fig_senadores.add_trace(go.Bar(
    x=valor_total_senador_2019.sort_values(by='valor_total', ascending=False).head(10)['senador'],
    y=valor_total_senador_2019.sort_values(by='valor_total', ascending=False).head(10)['valor_total'],
    marker_color='purple', name="", hovertemplate='R$ %{y:,.2f}', texttemplate='R$ %{y:,.2s}'
), row=2, col=1)
fig_senadores.add_trace(go.Bar(
    x=valor_total_senador_2020.sort_values(by='valor_total', ascending=False).head(10)['senador'],
    y=valor_total_senador_2020.sort_values(by='valor_total', ascending=False).head(10)['valor_total'],
    marker_color='purple', name="", hovertemplate='R$ %{y:,.2f}', texttemplate='R$ %{y:,.2s}'
), row=3, col=1)
fig_senadores.add_trace(go.Bar(
    x=valor_total_senador_2021.sort_values(by='valor_total', ascending=False).head(10)['senador'],
    y=valor_total_senador_2021.sort_values(by='valor_total', ascending=False).head(10)['valor_total'],
    marker_color='purple', name="", hovertemplate='R$ %{y:,.2f}', texttemplate='R$ %{y:,.2s}'
), row=4, col=1)
fig_senadores.add_trace(go.Bar(
    x=valor_total_senador_2022.sort_values(by='valor_total', ascending=False).head(10)['senador'],
    y=valor_total_senador_2022.sort_values(by='valor_total', ascending=False).head(10)['valor_total'],
    marker_color='purple', name="", hovertemplate='R$ %{y:,.2f}', texttemplate='R$ %{y:,.2s}'
), row=5, col=1)
fig_senadores.update_xaxes(title_font_size=14, tickfont_size=14)
fig_senadores.update_yaxes(title_font_size=14, tickfont_size=14)
fig_senadores.update_traces(textposition='inside', textfont_size=12)
fig_senadores.update_layout(
    font_family="Arial",
    hoverlabel_font_family="Arial",
    title_text="Despesas dos GASTOS dos Senadores por ANO",
    hoverlabel_font_size=14,
    showlegend=False,
    autosize=False,
    height=1600, 
    width=800,
    font=dict(
        size=14,
        color="purple"
    ),
)
fig_senadores.show(renderer='colab')